In [63]:
import os,time,sys,subprocess,warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
import tensorflow as tf
import tensorflow_datasets as tfds
from pathlib import Path
from tensorflow import keras
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "plotly_dark"
tf.get_logger().setLevel("ERROR")

# Creating the Training Dataset

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
shake = keras.utils.get_file("shakes.txt",'https://homl.info/shakespeare')
with open(shake) as f:
    shakes = f.read()

In [4]:
print(shakes[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


- encode every character to integer
  - one option to create custom preprocessing layer
  - second is to use keras tokenizer class

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakes)
tokenizer.texts_to_sequences(["Hello this is mario"])
print("".join(tokenizer.sequences_to_texts(np.random.randint(1,50,size=[1,50]))))
max_id = len(tokenizer.word_index)
total_size = tokenizer.document_count
encoded = np.array(tokenizer.texts_to_sequences(shakes))-1
encoded[:5]

s h ; ? u b . $ : a j j e m $ t o j , s s ' r - y ! : o & - m ' h ! j 3 a m ? 



array([[19],
       [ 5],
       [ 8],
       [ 7],
       [ 2]])

In [6]:
token_layer = keras.layers.TextVectorization(split="character",standardize="lower")
token_layer.adapt([shakes])
encoded = token_layer([shakes])[0]
encoded -= 2

In [7]:
n_tokens = token_layer.vocabulary_size()-2

In [8]:
train_size = total_size * 90//100
valid_size = total_size - train_size
print(f"train size = {train_size} and valid size = {valid_size}")

train size = 1003854 and valid size = 111540


In [9]:
n_steps = 100
batch_size = 32
shuffle_size = int(1e4)

In [10]:
tf.config.list_logical_devices()
strategy = tf.distribute.OneDeviceStrategy(device="/device:GPU:0")

In [11]:
def create_ds(slices,seq_len=100,batch_size=32,shuffle_size=10000,shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.window(seq_len+1,shift=1,drop_remainder=True).flat_map(lambda x: x.batch(seq_len+1))
    if shuffle:
        ds = ds.shuffle(shuffle_size)
    ds = ds.batch(batch_size)
    ds = ds.map(lambda x: (x[:,:-1],x[:,1:]))
    return ds.prefetch(1)

In [12]:
train_ds = create_ds(encoded[:train_size],shuffle=True)
valid_ds = create_ds(encoded[train_size:train_size+80_000])
test_ds = create_ds(encoded[train_size+80_000:])

In [13]:
# with strategy.scope():
#     model = keras.Sequential()
#     model.add(keras.layers.Embedding(input_dim=max_id,output_dim=16))
#     model.add(keras.layers.GRU(128,return_sequences=True))
#     model.add(keras.layers.Dense(max_id,"softmax"))
#     model.compile(
#         loss="sparse_categorical_crossentropy",
#         optimizer="nadam",
#         metrics=["accuracy"]
#     )
# checkp = keras.callbacks.ModelCheckpoint("shakes_model",monitor="val_accuracy",save_best_only=True)
# model.fit(train_ds,validation_data=valid_ds,epochs=10,callbacks=[checkp])
'''This will take about an hour so skipping and downloading the model'''

'This will take about an hour so skipping and downloading the model'

In [14]:
url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = keras.utils.get_file("shakespeare_model.tgz",origin=url,extract=True)
model_path = Path(path).with_name("shakespeare_model")
shakes_model = keras.models.load_model(model_path)

In [15]:
phrase = "To be or not to b"
tf.argmax(shakes_model.predict([phrase])[0,-1])+2

1/1 [==============================] - 2s 2s/step


<tf.Tensor: shape=(), dtype=int64, numpy=6>

In [32]:
def generate_poem(phrase,using_model=shakes_model,chars=50,temperature=1):
    for i in range(chars):
        logits = tf.math.log(using_model.predict([phrase],verbose=0)[0,-1:])
        rescaled_logits = logits/temperature
        char_ids = tf.reshape(tf.random.categorical(rescaled_logits,num_samples=1),shape=[])
        phrase += token_layer.get_vocabulary()[char_ids+2]

    print(phrase)
tf.random.set_seed(42)
generate_poem("To be or not to b",temperature=0.01)

To be or not to bai tcei esttletylstctei tcitnaywrsttctitoheai
eeao


In [50]:
def next_char(text,using_model,temperature=1):
    y_proba = using_model.predict([text],verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return token_layer.get_vocabulary()[char_id + 2]

def extend_text(text, n_chars=50,using_model=shakes_model, temperature=1):
    for _ in range(n_chars):
        text += next_char(text,using_model=shakes_model, temperature=temperature)
    return text

tf.random.set_seed(42)
print(extend_text("To be or not to b"))

To be or not to b l,uk
oaioaeaeslm
 l!oextl,ywxld.oeitoaieesdnuxers


# Stateful RNN

In [17]:
def create_stateful_ds(slices,seq_len=100):
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.window(seq_len+1,shift=seq_len,drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(seq_len+1))
    ds = ds.batch(1)
    ds = ds.map(lambda x: (x[:,:-1],x[:,1:])).prefetch(1)
    return ds

In [18]:
train_size = 1_000_000
train_ds = create_stateful_ds(encoded[:train_size])
valid_ds = create_stateful_ds(encoded[train_size:train_size+70_000])
test_ds = create_stateful_ds(encoded[train_size+70_000:])

In [19]:
stateful_shakes = keras.Sequential([
    keras.layers.Embedding(input_dim=n_tokens,output_dim=16,batch_input_shape=[1,None]),
    keras.layers.GRU(256,return_sequences=True,stateful=True),
    keras.layers.Dense(n_tokens,activation="softmax")
])
stateful_shakes.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)
class ResetStates(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.reset_states()

In [20]:
history = stateful_shakes.fit(train_ds,validation_data=valid_ds,epochs=10,callbacks=[ResetStates()])

Epoch 1/10
9999/9999 [==============================] - 101s 10ms/step - loss: 1.7648 - accuracy: 0.4764 - val_loss: 1.5974 - val_accuracy: 0.5218
Epoch 2/10
9999/9999 [==============================] - 96s 10ms/step - loss: 1.4568 - accuracy: 0.5558 - val_loss: 1.5130 - val_accuracy: 0.5476
Epoch 3/10
9999/9999 [==============================] - 97s 10ms/step - loss: 1.3788 - accuracy: 0.5764 - val_loss: 1.4841 - val_accuracy: 0.5564
Epoch 4/10
9999/9999 [==============================] - 97s 10ms/step - loss: 1.3388 - accuracy: 0.5864 - val_loss: 1.4702 - val_accuracy: 0.5611
Epoch 5/10
9999/9999 [==============================] - 96s 10ms/step - loss: 1.3144 - accuracy: 0.5930 - val_loss: 1.4631 - val_accuracy: 0.5616
Epoch 6/10
9999/9999 [==============================] - 96s 10ms/step - loss: 1.2983 - accuracy: 0.5974 - val_loss: 1.4620 - val_accuracy: 0.5632
Epoch 7/10
9999/9999 [==============================] - 97s 10ms/step - loss: 1.2882 - accuracy: 0.5992 - val_loss: 1.4596 

In [28]:
# shakes_stateful_model = keras.models.load_model("shakes_stateful_model")

In [29]:
shakes_stateless_model = keras.Sequential([
    keras.layers.Embedding(input_dim=n_tokens,output_dim=16),
    keras.layers.GRU(256,return_sequences=True),
    keras.layers.Dense(n_tokens,"softmax")
])

In [30]:
shakes_stateless_model.build(input_shape=tf.TensorShape([None,None]))

In [31]:
shakes_stateless_model.set_weights(stateful_shakes.get_weights())

In [61]:
full_model = keras.Sequential([
    token_layer,
    keras.layers.Lambda(lambda x: x-2),
    shakes_stateless_model
])

In [62]:
print(extend_text("To be or not to b",using_model=stateful_shakes,temperature=1))

To be or not to bohaiy
tcci
eslsttrlnxldgrenuxtcheitceesdi tcescei.
